# Logistic Regression

- binary problem. p(x = 1) = 1 - p(x=0)

- sigmoid function: $\sigma(x) = \frac{1}{1 + e ^{-x}}$ x가 w

- Hypothesis: $H(x) = \frac{1}{1 + e ^{-W^{T}X}}$ = $p(x=1;w) = 1-p(x=0;w) $
- Weight Update via GD: $W := W - a \frac{\delta}{\delta W}cost(W)$

- Cost: $cost(W) = -\frac{1}{m}\sum y \log(H(x))+(1-y)(\log(1- H(x))$

# import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
## Train data
x_data = [[1,2], [2,3], [3,1], [4,3], [5,3], [6,2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [3]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
x_train.shape, y_train.shape

(torch.Size([6, 2]), torch.Size([6, 1]))

In [5]:
# pytorch에서 e는 torch.exp로 사용 가능
print('e^1 = ', torch.exp(torch.FloatTensor([1])))

e^1 =  tensor([2.7183])


In [6]:
w = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# hypothsis

In [7]:
hypothesis = 1/(1+ torch.exp(-(x_train.matmul(w)+b)))
## 혹은 torch에서 제공하는 sigmoid 함수를 쓰는 것도 가능하다.
hypothesis = torch.sigmoid(x_train.matmul(w) + b)

In [8]:
print(hypothesis)
print(hypothesis.shape)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [21]:
-(y_train[0] * torch.log(hypothesis[0])) +   (1 - y_train[0]) * torch.log(1 - hypothesis[0])

tensor([-0.0003], grad_fn=<AddBackward0>)

In [10]:
# 하나의 값일때 다음과 같이 계산한다.
               # log p(x = 1 ; w)                         log p(x = 0; w)
-(y_train[0]* torch.log(hypothesis[0]))+ (1-y_train[0]) * torch.log(1 - hypothesis[0])

tensor([-0.6931], grad_fn=<AddBackward0>)

In [11]:
losses = -(y_train* torch.log(hypothesis)+ (1-y_train) * torch.log(1 - hypothesis))

In [12]:
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [13]:
## mean을 사용해서 개별의 losses를 평균해준다.
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [14]:
## binary cross entropy를 통해서 구할 수 있음.
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

# 전체 모델 작성하기.

## Training with Low-level Binary Cross Entropy Loss

In [15]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Training with F.binary_cross_entropy

In [16]:
# init model
w = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer사용
optimizer = optim.SGD([w, b], lr = 1)
nb_epochs = 1000

for epoch in range(nb_epochs +1):
    hypothesis = torch.sigmoid(x_train.matmul(w) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x)개선하기
    optimizer.zero_grad() # gradient 초기화
    cost.backward() # cost에 back prop수행.
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {} / {}, {}'.format(epoch, nb_epochs, cost.item()))

Epoch 0 / 1000, 0.6931471824645996
Epoch 100 / 1000, 0.13472206890583038
Epoch 200 / 1000, 0.0806431770324707
Epoch 300 / 1000, 0.05790002644062042
Epoch 400 / 1000, 0.0452997200191021
Epoch 500 / 1000, 0.03726094961166382
Epoch 600 / 1000, 0.03167250379920006
Epoch 700 / 1000, 0.027555905282497406
Epoch 800 / 1000, 0.024394338950514793
Epoch 900 / 1000, 0.02188830077648163
Epoch 1000 / 1000, 0.019852152094244957


# evaluation

In [17]:
hypothesis = torch.sigmoid(x_train.matmul(w)+ b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [18]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5].float())
print(y_train[:5])

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [19]:
correct_prediction = prediction.float() == y_train
print(correct_prediction.float())

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


In [20]:
accuracy = correct_prediction.sum().items/ len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

AttributeError: 'Tensor' object has no attribute 'items'

# nn.module을 사용한 더 개선된 표현

In [ ]:
import numpy as np

In [ ]:
Dropbox\모모딥\pytorch

In [ ]:
xy = np.loadtxt('~/Dropbox/모모딥/pytorch/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(6, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [ ]:
model = BinaryClassifier()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 100

for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        predict = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))